In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import poisson 
import math
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn import preprocessing
from pretty_html_table import build_table

def sum_of_lower_upper(mat):
    r = len(mat)
    c = len(mat[0])
    diag = np.trace(mat)
    lower_sum = 0
    upper_sum = 0
    over_sum = 0
    under_sum = 0
    gg_sum = 0
    ng_sum = 0 

    for i in range(r):
        for j in range(c):
            if (i<j):
                upper_sum += mat[i][j]

    for i in range(r):
        for j in range(c):
            if (j<i):
                lower_sum += mat[i][j]
    
    for i in range(r):
        for j in range(c):
            if (i+j>=3):
                over_sum += mat[i][j]
            else:
                under_sum += mat[i][j]
    
    for i in range(r):
        for j in range(c):
            if (i==0) or (j==0):
                ng_sum += mat[i][j]
            else:
                gg_sum += mat[i][j]

    return (upper_sum, lower_sum, diag, over_sum, under_sum, ng_sum, gg_sum)


## Gathering Lamda from season long performance

In [2]:
df1 = pd.read_csv('GatheredData/Full20xxFiles/Full2017.csv')
df2 = pd.read_csv('GatheredData/Full20xxFiles/Full2018.csv')
df3 = pd.read_csv('GatheredData/Full20xxFiles/Full2019.csv')
df4 = pd.read_csv('GatheredData/Full20xxFiles/Full2020.csv')
df5 = pd.read_csv('GatheredData/Full20xxFiles/Full2021.csv')
df6 = pd.read_csv('GatheredData/Full20xxFiles/Full2022.csv')
PRED = pd.read_csv('UpcomingFixtures/Prediction.csv')
pred = PRED.copy()
#Keeping bookmakers odds
Odds = PRED.copy()
Odds = PRED[['HTeam', 'ATeam','B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5', 'GG', 'NG']]

df1 = df1[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df2 = df2[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df3 = df3[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df3.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df3.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df4 = df4[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df4.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df4.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df5 = df5[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df5.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df5.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df6 = df6[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]

df1.rename(columns = {'Date_x':'Date'}, inplace = True)
df2.rename(columns = {'Date_x':'Date'}, inplace = True)
df3.rename(columns = {'Date_x':'Date'}, inplace = True)
df4.rename(columns = {'Date_x':'Date'}, inplace = True)
df5.rename(columns = {'Date_x':'Date'}, inplace = True)


Season = pd.read_csv('GatheredData/Full20xxFiles/Full2023.csv')
Season = Season[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
Season = Season.iloc[38:]


DF = pd.concat([df1, df2, df3, df4, df5, df6, Season], ignore_index=True)


DF = DF[ DF['HM'] != 0] 
DF = DF[ DF['HM'] != 1] 
DF = DF[ DF['HM'] != 2] 
DF = DF[ DF['HM'] != 3]
subset = DF[['Date', 'HTeam', 'ATeam']].copy()

extra_stats = DF.copy()
extra_pred = pred.copy()

DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']
pred['PTSDIF/M']=pred['HPTS/M']-pred['APTS/M']
DF = DF[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
pred = pred[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
cut = DF.shape[0]
print(cut)
DF = pd.concat([DF, pred], ignore_index=True)
try:
    DF['HxG'] = DF['HxG'].div(DF['HM'])
    DF['HG'] = DF['HG'].div(DF['HM'])
    DF['HGA'] = DF['HGA'].div(DF['HM'])
    DF['HxGA'] = DF['HxGA'].div(DF['HM'])
    DF['AxG'] = DF['AxG'].div(DF['AM'])
    DF['AG'] = DF['AG'].div(DF['AM'])
    DF['AxGA'] = DF['AxG'].div(DF['AM'])
    DF['AGA'] = DF['AGA'].div(DF['AM'])
    pred['HxG'] = pred['HxG'].div(pred['HM'])
    pred['HG'] = pred['HG'].div(pred['HM'])
    pred['HGA'] = pred['HGA'].div(pred['HM'])
    pred['HxGA'] = pred['HxGA'].div(pred['HM'])
    pred['AxG'] = pred['AxG'].div(pred['AM'])
    pred['AG'] = pred['AG'].div(pred['AM'])
    pred['AxGA'] = pred['AxG'].div(pred['AM'])
    pred['AGA'] = pred['AGA'].div(pred['AM'])

except ZeroDivisionError:
    DF['HxG'] = 0
    DF['HG'] = 0
    DF['HGA'] = 0
    DF['HxGA'] = 0
    DF['AxG'] = 0
    DF['AG'] = 0
    DF['AxGA'] = 0
    DF['AGA'] = 0

HGoals = []
AGoals = []
    
for i, x in DF.iloc[:cut].iterrows():
    a=x['Result'].split('\'')
    HGoals.append(a[1])
    AGoals.append(a[3])
    
DF=np.array(DF)
DFnorm = preprocessing.normalize(DF[:, 0:21], axis=0)
pred = DFnorm[cut:]
DF = DFnorm[:cut]

x=DF


y1=np.array(HGoals)
y2=np.array(AGoals)

reg1 = LinearRegression().fit(x, y1)
reg2 = LinearRegression().fit(x, y2)

LamdaHome= reg1.predict(pred)
LamdaAway= reg2.predict(pred)

2082


## Gathering Lamda from 2 month form performance

In [3]:
df1 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2017Monthly.csv')
df2 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2018Monthly.csv')
df3 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2019Monthly.csv')
df4 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2020Monthly.csv')
df5 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2021Monthly.csv')
df6 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2022Monthly.csv')
pred = PRED.copy()
#Keeping bookmakers odds
Odds = PRED.copy()
Odds = PRED[['HTeam', 'ATeam','B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5', 'GG', 'NG']]

df1 = df1[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df2 = df2[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df3 = df3[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df3.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df3.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
df4 = df4[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df4.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df4.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
df5 = df5[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df5.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df5.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
df6 = df6[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df6.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df6.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)

SeasonForm = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2023Monthly.csv')


try:
    SeasonForm = SeasonForm[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
except KeyError:
    SeasonForm = SeasonForm[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
    SeasonForm.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
    SeasonForm.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
    
SeasonForm = SeasonForm.iloc[38:]

DF = pd.concat([df1, df2, df3, df4, df5, df6, SeasonForm], ignore_index=True)
merged = pd.merge(subset, DF, on=['Date', 'HTeam', 'ATeam'], how='left')
print(subset.iloc[1361])
print(DF.iloc[1361])
print(merged[merged.isna().any(axis=1)])

# resulting dataframe will have columns of B where the triplet of A['a', 'b', 'c'] exists in B
DF = merged[DF.columns]

DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']
pred['PTSDIF/M']=pred['HPTS/M']-pred['APTS/M']
DF = DF[['HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
pred = pred[['HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
cut = DF.shape[0]
DF = pd.concat([DF, pred], ignore_index=True)



try:
    DF['HxG'] = DF['HxG'].div(DF['HM']).fillna(0)
    DF['HG'] = DF['HG'].div(DF['HM']).fillna(0)
    DF['HGA'] = DF['HGA'].div(DF['HM']).fillna(0)
    DF['HxGA'] = DF['HxGA'].div(DF['HM']).fillna(0)
    DF['AxG'] = DF['AxG'].div(DF['AM']).fillna(0)
    DF['AG'] = DF['AG'].div(DF['AM']).fillna(0)
    DF['AxGA'] = DF['AxG'].div(DF['AM']).fillna(0)
    DF['AGA'] = DF['AGA'].div(DF['AM']).fillna(0)
    pred['HxG'] = pred['HxG'].div(pred['HM']).fillna(0)
    pred['HG'] = pred['HG'].div(pred['HM']).fillna(0)
    pred['HGA'] = pred['HGA'].div(pred['HM']).fillna(0)
    pred['HxGA'] = pred['HxGA'].div(pred['HM']).fillna(0)
    pred['AxG'] = pred['AxG'].div(pred['AM']).fillna(0)
    pred['AG'] = pred['AG'].div(pred['AM']).fillna(0)
    pred['AxGA'] = pred['AxG'].div(pred['AM']).fillna(0)
    pred['AGA'] = pred['AGA'].div(pred['AM']).fillna(0)

except ZeroDivisionError:
    DF['HxG'] = 0
    DF['HG'] = 0
    DF['HGA'] = 0
    DF['HxGA'] = 0
    DF['AxG'] = 0
    DF['AG'] = 0
    DF['AxGA'] = 0
    DF['AGA'] = 0


print(DF.iloc[:cut])
        
HGoals = []
AGoals = []
for i, x in DF.iloc[:cut].iterrows():
    a=str(x['Result']).split('\'')
    try:
        HGoals.append(a[1])
        AGoals.append(a[3])
    except IndexError:
        print(x)
        print(DF.iloc[:cut].iloc[i-1])

DF=np.array(DF)

DFnorm = preprocessing.normalize(DF[:, 0:21].astype('float64'), axis=0)
pred = DFnorm[cut:]
DF = DFnorm[:cut]

x=DF

y1=np.array(HGoals)
y2=np.array(AGoals)

reg1 = LinearRegression().fit(x, y1)
reg2 = LinearRegression().fit(x, y2)

LamdaHomeForm= reg1.predict(pred)
LamdaAwayForm= reg2.predict(pred)

Date           17/09/2021
HTeam    Newcastle United
ATeam               Leeds
Name: 1560, dtype: object
Date                     07/02/2021
HxG                           12.05
HM                               12
HTeam       Wolverhampton Wanderers
ATeam                     Leicester
HG                               12
HGA                              20
HxGA                          15.48
HDC                        3.166667
HODC                       7.083333
HPPDA                         15.93
HOPPDA                        11.56
HPTS/M                         0.75
AM                               12
AG                               20
AGA                              11
AxG                           16.59
AxGA                          10.23
ADC                             7.0
AODC                       4.333333
APPDA                           9.7
AOPPDA                        12.24
APTS/M                          2.0
Result                 ['0 ', ' 0']
B365H                           

/tmp/ipykernel_190845/2412402004.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']


In [4]:
LamdaAwayFinal = [(LamdaAway[i] * 0.4 + LamdaAwayForm[i] * 0.6) for i in range(len(LamdaAway))]
LamdaHomeFinal = [(LamdaHome[i] * 0.4 + LamdaHomeForm[i] * 0.6) for i in range(len(LamdaHome))]

print(LamdaHomeFinal)
print(LamdaAwayFinal)

[1.8218833361032514, 2.5199534347386683, 1.1942965293819583, 2.0727631473758756, 2.724677514037657, 1.6234682717156366, 1.085934836695626, 1.4815385117051063, 2.182997832735592, 1.9624813260235605, 2.2337283666804932, 1.4762278725738824, 1.31071564654391, 1.8336806975514712, 1.3276415809824549, 2.8354162133094576, 1.4123866206484355, 1.0976387086903832, 1.6336999251357138, 1.205454639091689, 0.8515507488544705, 2.1101464283726252]
[1.1943923703278676, 1.179283534162096, 2.0518308944452364, 1.2684551933794468, 0.5183672060527156, 1.6464101370017779, 2.093064328291354, 1.4712842050581973, 1.202545651548399, 0.9577108389393706, 0.8861389674742223, 1.6778236053547326, 1.5088512175061797, 1.2595513954670432, 1.2809844345814203, 0.49180196282642474, 1.7154387023807687, 1.702815354538235, 1.4222147297471197, 1.5479968963127089, 2.149629128481617, 1.5837440184750913]


In [5]:
k = range(12)
PH = np.zeros((len(k), 1))
PA = np.zeros((1, len(k)))
PRED['P(Hwin)']= np.nan
PRED['P(Awin)']= np.nan
PRED['P(Draw)']= np.nan
PRED['P(Over)']= np.nan
PRED['P(Under)']= np.nan
PRED['P(GG)']=np.nan
PRED['P(NG)']=np.nan

Pie_list = []
for j, l in enumerate(LamdaHome):
    for i in k:
        PH[i, 0] = (math.exp(-l)*l**k[i]) /math.factorial(k[i])
        PA[0, i] = (math.exp(-LamdaAway[j])*LamdaAway[j]**k[i]) /math.factorial(k[i])
    Pscore = np.matmul(PH, PA) 
    df = pd.DataFrame(Pscore, columns = ['0','1','2', '3', '4', '5', '6', '7', '8', '9', '10', '11'])
    PRED.loc[j,'P(Awin)'], PRED.loc[j,'P(Hwin)'], PRED.loc[j,'P(Draw)'], PRED.loc[j,'P(Over)'], PRED.loc[j,'P(Under)'], PRED.loc[j,'P(NG)'], PRED.loc[j,'P(GG)']  = sum_of_lower_upper(Pscore)
    PRED['P(Over)'] = 1 - PRED['P(Under)']
    df = df.iloc[0:5, 0:5]
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    
    G = []
    for i in range(0, 5):
        for j in range(0, 5):
            G.append(df.loc[i][j])

            
    Pie_Df = pd.DataFrame(G, columns = ['P(score)'])
    Pie_Df['score']= ['0 - 0',
                      '0 - 1',
                      '0 - 2',
                      '0 - 3',
                      '0 - 4',
                      '1 - 0',
                      '1 - 1',
                      '1 - 2',
                      '1 - 3',
                      '1 - 4',
                      '2 - 0',
                      '2 - 1',
                      '2 - 2',
                      '2 - 3',
                      '2 - 4',
                      '3 - 0',
                      '3 - 1',
                      '3 - 2',
                      '3 - 3',
                      '3 - 4',
                      '4 - 0',
                      '4 - 1',
                      '4 - 2',
                      '4 - 3',
                      '4 - 4']
    Pie_Df.loc[Pie_Df["P(score)"] < 0.02, "score"] = 'Other'
    Pie_Df = Pie_Df.sort_values(by=['P(score)'], ascending=False)
    Pie_list.append(Pie_Df)
    

PRED = PRED[['Date', 'HTeam', 'ATeam', 'P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Under)', 'P(Over)', 'P(GG)', 'P(NG)']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(PRED)

pd.options.display.float_format = '{:.2f}'.format
disp_pred = PRED.copy()
disp_pred.rename(columns = {'ATeam':'Away Team'}, inplace = True)
disp_pred.rename(columns = {'HTeam':'Home Team'}, inplace = True)
html_table_blue_light = build_table(disp_pred, 'blue_light')

# Save to html file
with open('UpcomingMatchesPrediction.html', 'w') as f:
    f.write(html_table_blue_light)

json_data = disp_pred.to_json()
with open('ProbabilityPrediction.json', 'w') as f:
    f.write(json_data)

Final_Odds = pd.concat([PRED, Odds], axis=0)
Final_Odds['Type'] = np.nan
Final_Odds['Match'] = np.nan
Final_Odds = Final_Odds.reset_index(drop = 'True')
Final_Odds.loc[Final_Odds['B365H'].isnull(), 'Type'] = 'Predicted'
Final_Odds.loc[Final_Odds['P(Hwin)'].isnull(), 'Type'] = 'Bookmaker'

for i, x in enumerate(Final_Odds['Match']):
    Final_Odds['Match'][i] = str(Final_Odds['HTeam'][i] + '-' + Final_Odds['ATeam'][i])

li  = Final_Odds.index[Final_Odds['Type'] == 'Bookmaker'].tolist()
for i in li:
    Final_Odds['P(Hwin)'][i] = 1/Final_Odds.iloc[i]['B365H']
    Final_Odds['P(Awin)'][i] = 1/Final_Odds.iloc[i]['B365A']
    Final_Odds['P(Draw)'][i] = 1/Final_Odds.iloc[i]['B365D']
    Final_Odds['P(Over)'][i] = 1/Final_Odds.iloc[i]['BbAv>2.5']
    Final_Odds['P(Under)'][i] = 1/Final_Odds.iloc[i]['BbAv<2.5']
    Final_Odds['P(GG)'][i] = 1/Final_Odds.iloc[i]['GG']
    Final_Odds['P(NG)'][i] = 1/Final_Odds.iloc[i]['NG']

Final_Odds = Final_Odds[['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Under)', 'P(Over)', 'P(GG)', 'P(NG)', 'Type', 'Match', 'Date']].copy()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(Final_Odds)

    

,Date,HTeam,ATeam,P(Hwin),P(Draw),P(Awin),P(Under),P(Over),P(GG),P(NG)
0,22/10/2023,Aston Villa,West Ham,0.544,0.223,0.233,0.407,0.593,0.587,0.413
1,23/10/2023,Tottenham,Fulham,0.712,0.158,0.130,0.271,0.729,0.619,0.381
2,27/10/2023,Crystal Palace,Tottenham,0.221,0.209,0.570,0.357,0.643,0.620,0.380
3,28/10/2023,Chelsea,Brentford,0.569,0.210,0.221,0.359,0.641,0.619,0.381
4,28/10/2023,Arsenal,Sheffield United,0.873,0.096,0.031,0.369,0.631,0.309,0.691
5,28/10/2023,Bournemouth,Burnley,0.385,0.233,0.381,0.376,0.624,0.640,0.360
6,28/10/2023,Wolverhampton Wanderers,Newcastle United,0.171,0.201,0.628,0.396,0.604,0.558,0.442
7,29/10/2023,West Ham,Everton,0.402,0.241,0.358,0.414,0.586,0.610,0.390
8,29/10/2023,Brighton,Fulham,0.614,0.196,0.190,0.338,0.662,0.616,0.384
9,29/10/2023,Aston Villa,Luton,0.641,0.201,0.157,0.420,0.580,0.529,0.471


/tmp/ipykernel_190845/2946263194.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['Match'][i] = str(Final_Odds['HTeam'][i] + '-' + Final_Odds['ATeam'][i])
/tmp/ipykernel_190845/2946263194.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['P(Hwin)'][i] = 1/Final_Odds.iloc[i]['B365H']
/tmp/ipykernel_190845/2946263194.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['P(Awin)'][i] = 1/Final_Odds.

,P(Hwin),P(Draw),P(Awin),P(Under),P(Over),P(GG),P(NG),Type,Match,Date
0,0.54,0.22,0.23,0.41,0.59,0.59,0.41,Predicted,Aston Villa-West Ham,22/10/2023
1,0.71,0.16,0.13,0.27,0.73,0.62,0.38,Predicted,Tottenham-Fulham,23/10/2023
2,0.22,0.21,0.57,0.36,0.64,0.62,0.38,Predicted,Crystal Palace-Tottenham,27/10/2023
3,0.57,0.21,0.22,0.36,0.64,0.62,0.38,Predicted,Chelsea-Brentford,28/10/2023
4,0.87,0.10,0.03,0.37,0.63,0.31,0.69,Predicted,Arsenal-Sheffield United,28/10/2023
5,0.39,0.23,0.38,0.38,0.62,0.64,0.36,Predicted,Bournemouth-Burnley,28/10/2023
6,0.17,0.20,0.63,0.40,0.60,0.56,0.44,Predicted,Wolverhampton Wanderers-Newcastle United,28/10/2023
7,0.40,0.24,0.36,0.41,0.59,0.61,0.39,Predicted,West Ham-Everton,29/10/2023
8,0.61,0.20,0.19,0.34,0.66,0.62,0.38,Predicted,Brighton-Fulham,29/10/2023
9,0.64,0.20,0.16,0.42,0.58,0.53,0.47,Predicted,Aston Villa-Luton,29/10/2023


In [6]:
fig = make_subplots(rows=2, cols=1)
fig.update_layout(width=1500, height=1000)
fig.add_trace(
    go.Bar(name='Predicted Home Win',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Hwin)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Home Win',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Hwin)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Predicted Draw',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Draw)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Draw',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Draw)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Predicted Away Win',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Awin)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Away Win',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Awin)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)


fig.add_trace(
    go.Bar(name='Predicted Over',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Over)'],
           marker_color='rgb(141,211,199)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Over',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Over)'],
           marker_color='rgb(82, 106, 131)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Predicted Under',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Under)'],
           marker_color='rgb(141,211,199)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Under',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Under)'],
           marker_color='rgb(82, 106, 131)'),
    row=2, col=1)


fig.show()


In [7]:
# Create subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'xy'}]], column_widths=[0.33, 0.67])
fig.update_layout(width=1500, height=650)
# create a list to store the buttons
buttons = []
categories = ['Home', 'Draw', 'Away', 'Over', 'Under', 'GG', 'NG']

# loop through each match and create two traces for Bookmaker odds and predicted values
for match_index, match in enumerate(Final_Odds['Match'].unique().tolist()):

    # create the trace for Bookmaker odds
    trace_bookmaker = go.Scatterpolar(
      r=Final_Odds[(Final_Odds['Type']=='Bookmaker')  & (Final_Odds['Match']==match)][['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Over)', 'P(Under)', 'P(GG)', 'P(NG)']].values.tolist()[0],
      theta=categories,
      fill='toself',
      name='Bookmaker Odds',
      marker = dict(color = 'rgb(82, 106, 131)' ),
      visible=False if match_index!=0 else True,
      hovertemplate='%{theta}<br>Prob: %{r:.2f}<br>Bookmaker Odds'
)
    fig.add_trace(trace_bookmaker, row=1, col=1)
    # create the trace for predicted values
    trace_predicted = go.Scatterpolar(
        r= Final_Odds[(Final_Odds['Type']=='Predicted')  & (Final_Odds['Match']==match)][['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Over)', 'P(Under)', 'P(GG)', 'P(NG)']].values.tolist()[0],
        theta=categories,
        fill='toself',
        name='Predicted Odds',
        marker = dict(color ='rgb(141,211,199)'), #'rgb(217, 175, 107)'
        visible=False if match_index!=0 else True,
        hovertemplate='%{theta}<br>Prob: %{r:.2f}<br>Predicted Odds'
  )
    fig.add_trace(trace_predicted, row=1, col=1)
    
    trace_goals = go.Bar(
        x=Pie_list[match_index]['score'],
        y=Pie_list[match_index]['P(score)'],
        visible=False if match_index!=0 else True,
        marker=dict(color=Pie_list[match_index]['P(score)'], colorscale='darkmint'),
        name='Propable Scoreline',
        hovertemplate='Score %{x}<br>Prob: %{y:.3f}'
    )
    fig.add_trace(trace_goals, row=1, col=2)
    
    # create the dictionary for the button
    button_dict = dict(
        method="restyle",
        args=[{"visible": [False] * (len(Final_Odds['Match'].unique().tolist()) * 3)}], # set all traces to false initially
        label= f"<b>{match} | {str(Final_Odds[(Final_Odds['Type']=='Predicted')  & (Final_Odds['Match']==match)]['Date'].values).strip('[]')}</b>" 
    )
    
    # set visibility to true for the corresponding traces
    button_dict["args"][0]["visible"][3*match_index] = True # Bookmaker odds trace
    button_dict["args"][0]["visible"][3*match_index+1] = True # Predicted trace
    button_dict["args"][0]["visible"][3*match_index+2] = True # Goal trace
    
    # set visibility to true for the corresponding traces for the first match
    if match_index == 0:
        button_dict["args"][0]["visible"][0] = True # Bookmaker odds trace
        button_dict["args"][0]["visible"][1] = True # Predicted trace
        button_dict["args"][0]["visible"][2] = True
        
    # append the button to the list
    buttons.append(button_dict)


# update the layout with the buttons
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        buttons=buttons,
        active=0, # set the initial active button
        x=0.1,
        y=1.2,
        direction="down",
    )],
    
)

# show the plot
fig.show()
# Save the interactive figure as an HTML file
fig.write_html('UpcomingMatchesPredictionFigure.html')


In [8]:
images_path = '/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheredData/Logos'
team_color = {"Liverpool": "red",
              "West Ham": "brown",
              "Brentford": "#ff8d33",
              "Everton": "blue",
              "Crystal Palace": "#1711e0",
              "Fulham": "black",
              "Luton": "#e0ae11",
              "Wolverhampton Wanderers": "#e09211",
              "Manchester City": "#0bdeef",
              "Nottingham Forest": "#ef2a0b",
              "Burnley": "#8e2b4e",
              "Manchester United": "#d12f07",
              "Arsenal": "#ff0101",
              "Tottenham": "#a5bfcc",
              "Brighton": "#228bbf",
              "Bournemouth": "#e50e21",
              "Chelsea": "#0e1ee5",
              "Aston Villa": "#a4266f",
              "Sheffield United": "#e31343",
              "Newcastle United": "#d5c0c5"}


In [9]:
import os
from PIL import Image

df = disp_pred



In [10]:
Pont = 5
bank = 60
Final = Final_Odds[Final_Odds['Type']=='Predicted'][['Match']]
listPred = Final_Odds[Final_Odds['Type']=='Predicted']['P(Under)'].tolist()
listBook = Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Under)'].tolist()

diff = []
for i, p in enumerate(listPred):
    diff.append(listPred[i]-listBook[i])

Final['UnderBet'] = np.nan
Final['UnderBet_Kelly'] = np.nan
for i, d in enumerate(diff):
    if (0.7>=listPred[i]>=0.5) and (d>=0.005):
        Final['UnderBet'][i] = max(Pont*10*(d),0)
        Final['UnderBet_Kelly'][i] = max(0, bank*(listPred[i]- (1-listPred[i])/(1/listBook[i] -1)))
    else:
        Final['UnderBet'][i] = 0
        Final['UnderBet_Kelly'][i] = 0
        
############

    
display(Final)

/tmp/ipykernel_190845/1283783480.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_190845/1283783480.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Match,UnderBet,UnderBet_Kelly
0,Aston Villa-West Ham,0.00,0.00
1,Tottenham-Fulham,0.00,0.00
2,Crystal Palace-Tottenham,0.00,0.00
3,Chelsea-Brentford,0.00,0.00
4,Arsenal-Sheffield United,0.00,0.00
5,Bournemouth-Burnley,0.00,0.00
6,Wolverhampton Wanderers-Newcastle United,0.00,0.00
7,West Ham-Everton,0.00,0.00
8,Brighton-Fulham,0.00,0.00
9,Aston Villa-Luton,0.00,0.00
